# Reading data

In this exercise we will cover how to use pandas to read data from external data sources. To perform our analysis, we will need to use two different data sets:

1. Business licenses data: <https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr>
2. Food inspections data: <https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5>

Both data sets are hosted on <https://data.cityofchicago.org>.

---

### Task 1 - read data

#### 🔄 Task

- Explore the two data sources.
- Can you load the data into pandas?

#### ✅ Solution

The City of Chicago data portal uses:

> The Socrata Open Data API (SODA) provides programmatic access to this dataset including the ability to filter, query, and aggregate data.

There is a Python package to interact with SODA, but it is no longer maintained: <https://github.com/xmunoz/sodapy>.

Instead of using the Python library, we can call the SODA API directly. Consulting the documentation provides us with some examples how how to use <https://dev.socrata.com/foundry/data.cityofchicago.org/4ijn-s7e5>.

For example, we can use `curl` to request the data in JSON form. This has a few problems though:

- The data is in JSON. We can work with this, but CSV may be more convenient.
- We are using the shell instead of Python




In [ ]:
%%sh
curl 'https://data.cityofchicago.org/resource/r5kz-chrr.json?$limit=10'

We can instead use Python and Pandas to make the request. Pandas has a built in method to read CSV data directly from a URL. So our first task will be to construct the URL in Python.

In [ ]:
from urllib.parse import urlencode


base_url = "https://data.cityofchicago.org"

# Note the .csv extension
path = "resource/r5kz-chrr.csv"

# To make our code easier to read we can define the parameters in a dict. To know
# what parameters are available you must consult the docs: https://dev.socrata.com/docs/queries/
params = {
    "$order": "id", 
    "$limit": 5
}

# Then use an f-string to construct the URL. You can use the built in urlencode
# function to correctly format the params.
url = f"{base_url}/{path}?{urlencode(params)}"
url

You can then pass in the newly constructed URL directly to Pandas.

In [ ]:
import pandas as pd

df = pd.read_csv(url)
df

---

### Task 2 - write data to SQL

#### 🔄 Task

- Save the raw data to the Postgres SQL database so that we do not need to hit the API every time we need to interact with the raw data.
- You can connect to the database using the following credentials:
  - host: `database.conf23workflows.training.posit.co`
  - user: `posit`
  - password: ???
  - database: `conf23_python`

🚨 Please prefix any tables you create with your name! For example:

- `sam_business_license_raw`
- `sam_food_inspections_raw`


#### ✅ Solution

There are many different ways to interact with SQL databases in Python. For writing data, we prefer to use [SQLAlchemy](https://docs.sqlalchemy.org/en/14/dialects/postgresql.html) with Pandas. You will need to make sure you have the following packages installed.

You will first need to create a connection to the database.

In [ ]:
os.environ["CONF23_DB_HOST"]
os.environ["CONF23_DB_PASSWORD"]

In [ ]:
import os

from sqlalchemy import create_engine, text

db_user = "posit"
db_password = os.environ["CONF23_DB_PASSWORD"]
db_host = os.environ["CONF23_DB_HOST"]
db_port = 5432
db_database = "conf23_python"
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_database}")
engine

You can then use pandas built in SQL functions to write data to SQL.

In [ ]:
df

In [ ]:
table_name = "samedwardes_business_license_test"

In [ ]:
df.to_sql(table_name, engine, if_exists="replace")

You can verify that it worked by reading the data from SQL:

In [ ]:
with engine.begin() as conn:
    data_from_sql = pd.read_sql(text(f"SELECT * FROM {table_name}"), conn)

data_from_sql

---

### Task 3 - put it all together

#### 🔄 Task

- Clean up your code from activity 1 to pull both datasets from the City of Chicago into pandas dataframes
- Write both dataframes into the Postgres database.

🚨 Writing data to postgres can be slow. Do not insert all of the rows in one go. Instead you should write a loop that inserts 10,000 rows at a time.


#### ✅ Solution

See [materials/example/01-etl-raw-data/notebook.ipynb](../example/01-etl-raw-data/notebook.ipynb) for a complete solution.

### Task 4 - Publish the notebook to Connect

#### 🔄 Task

Publish your Jupyter Notebook to Connect and schedule it to re-run every Sunday at 2:00 AM.

**Tips**

- You will need to create an API key.
  - Read this page to learn how to create an API key: https://docs.posit.co/connect/user/api-keys/
  - Export the API key as environment variable in your `~/.bashrc`
- Export a `CONNECT_SERVER` environment variable


```bash
# ~/.bashrc
export CONNECT_SERVER="https://connect.conf23workflows.training.posit.co"
export CONNECT_API_KEY="xxx"
```

Note that in this workshop, we have set `CONNECT_SERVER` as a global environment variable for everyone. You will still need to set your own API key. You can check what is already set by running:

```bash
env | grep CONNECT
```

#### ✅ Solution

```bash
cd materials/your-work/activity-1
rsconnect deploy notebook --title "01 - YOUR NAME - Raw Data ETL" notebook.ipynb
```